# Linear Regression

In this notebook, we will learn how to apply Linear regression for predicting the heating load requirements (Y1) of buildings as a function of building parameters (Xs).

The attached dataset is taken from the [UC Irvine Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Energy+efficiency).

To run this code, you will need the following python packages:
* numpy
* pandas
* openpyxl
* scikit-learn

In [36]:
import numpy as np
import pandas as pd

In [37]:
!pip install openpyxl

In [38]:
# First, we load the dataset using pandas
df = pd.read_excel("Energy_Efficiency.xlsx", engine = 'openpyxl')
# Remove any unnamed columns (might occur due to difference in pandas readers)
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
# Remove any row with NaNs
df = df.dropna(how='all')
# Drop Y2 (as we only consider Y1 for regression)
df = df.drop('Y2', axis=1)

In [39]:
# next, we will split the dataframe into a training and testing splits with a 70% / 30% ratio
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=0.3, random_state=42) # Random is fixed for reproducability

In [40]:
# Now lets display a few rows from the training data
df_train

,X1,X2,X3,X4,X5,X6,X7,X8,Y1
334,0.62,808.5,367.5,220.50,3.5,4,0.25,1,15.08
139,0.64,784.0,343.0,220.50,3.5,5,0.10,2,15.19
485,0.90,563.5,318.5,122.50,7.0,3,0.25,5,32.31
547,0.79,637.0,343.0,147.00,7.0,5,0.40,1,41.67
18,0.79,637.0,343.0,147.00,7.0,4,0.00,0,29.63
...,...,...,...,...,...,...,...,...,...
71,0.76,661.5,416.5,122.50,7.0,5,0.10,1,32.21
106,0.86,588.0,294.0,147.00,7.0,4,0.10,2,26.33
270,0.71,710.5,269.5,220.50,3.5,4,0.10,5,10.67
435,0.98,514.5,294.0,110.25,7.0,5,0.25,4,28.62


In [41]:
# Then lets view some statistics
df_train.describe()

,X1,X2,X3,X4,X5,X6,X7,X8,Y1
count,537.000000,537.000000,537.000000,537.000000,537.000000,537.000000,537.000000,537.000000,537.000000
mean,0.760354,674.867784,318.636872,178.115456,5.201117,3.500931,0.235940,2.854749,22.050503
std,0.104790,87.758133,43.619254,44.839207,1.750948,1.106502,0.134118,1.544532,10.088187
min,0.620000,514.500000,245.000000,110.250000,3.500000,2.000000,0.000000,0.000000,6.010000
25%,0.660000,612.500000,294.000000,147.000000,3.500000,3.000000,0.100000,2.000000,12.960000
50%,0.740000,686.000000,318.500000,220.500000,3.500000,3.000000,0.250000,3.000000,17.230000
75%,0.820000,759.500000,343.000000,220.500000,7.000000,4.000000,0.400000,4.000000,31.280000
max,0.980000,808.500000,416.500000,220.500000,7.000000,5.000000,0.400000,5.000000,43.100000


In [42]:
# Now we will extract the models input and targets from both the training and testing dataframes
def extract_Xy(df):
    df_numpy = df.to_numpy()
    return df_numpy[:, :-1], df_numpy[:, -1]

X_train, y_train = extract_Xy(df_train)
X_test, y_test = extract_Xy(df_test)

## Linear Regression via Scikit-Learn

In [43]:
# Then we test the linear regression using Scikit-learn's implementation
from sklearn.linear_model import LinearRegression

model = LinearRegression().fit(X_train, y_train)

In [44]:
# Using scikit-learn's MSE function, we can compute the training and testing error for our model
from sklearn.metrics import mean_squared_error

y_train_predict = model.predict(X_train)
training_error = mean_squared_error(y_train, y_train_predict)
print(f"Training Error: {training_error} (RMS: {training_error**0.5})")
y_test_predict = model.predict(X_test)
testing_error = mean_squared_error(y_test, y_test_predict)
print(f"Testing Error: {testing_error} (RMS: {testing_error**0.5})")

#Note: We also display the Root Mean Square error (RMS) since it is more intuitive to compare with the dataset statistics (diplayed using df_train.describe())


Training Error: 8.368261516136105 (RMS: 2.89279475873006)
Testing Error: 8.78499323427013 (RMS: 2.9639489257188845)


In [45]:
%%timeit
LinearRegression().fit(X_train, y_train)
# Here we are measuring the training time to compare with our implementation below

310 µs ± 6.19 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Linear Regression from Scratch

In [46]:
def our_mean_square_error(true, predicted):
    #TODO: implement this function to match Scikit-learn's mean_square_error
    #Note: both true & predicted will be float numpy arrays
    return np.mean((true - predicted)**2)

In [47]:
print(f"{our_mean_square_error( np.array([  1, 0]), np.array([1,   0]) ) = }") # Should be 0
print(f"{our_mean_square_error( np.array([  0, 1]), np.array([1,   0]) ) = }") # Should be 1
print(f"{our_mean_square_error( np.array([0.5, 0]), np.array([1, 0.5]) ) = }") # Should be 0.25

our_mean_square_error( np.array([  1, 0]), np.array([1,   0]) ) = 0.0
our_mean_square_error( np.array([  0, 1]), np.array([1,   0]) ) = 1.0
our_mean_square_error( np.array([0.5, 0]), np.array([1, 0.5]) ) = 0.25


In [48]:
class OurLinearRegression:
    def _prepare_inputs(self, X):
        # Here, we add a new input with value 1 to each example. It will be multipled by the bias
        ones = np.ones((X.shape[0], 1), dtype=X.dtype)
        return np.concatenate((ones, X), axis=1)

    def fit(self, X, y):
        X = self._prepare_inputs(X) # First, we prepare the inputs
        #TODO: compute and store the model weights into self.w
        # Note: you can use numpy function and do not use "numpy.linalg.lstsq" or "numpy.linalg.pinv"
        # To compute a square matrix's inverse, you can use "numpy.linalg.inv".
        # A more stable option to compute "numpy.linalg.inv(A) @ b" is using "numpy.linalg.solve(A, b)" 
        
        # to calculate the weights, we use the formula w = (X^T * X)^-1 * X^T * y
        # @ is the matrix multiplication operator        
        self.w = np.linalg.solve(X.T @ X, X.T @ y) 
        
        # Return self to match the behavior of Scikit-Learn's LinearRegression fit()
        return self
    
    def predict(self, X):
        X = self._prepare_inputs(X) # First, we prepare the inputs
        #TODO: Compute and return the predictions given X
        # use the model weights stored in self.w to compute the predictions 
        # @ is the matrix multiplication operator
        return X @ self.w

In [49]:
# Now, you can train your model
our_model = OurLinearRegression().fit(X_train, y_train)

In [50]:
# Using your MSE function, you can compute the training and testing error for our model
y_train_predict = our_model.predict(X_train)
training_error = our_mean_square_error(y_train, y_train_predict)
print(f"Training Error: {training_error} (RMS: {training_error**0.5})")
y_test_predict = our_model.predict(X_test)
testing_error = our_mean_square_error(y_test, y_test_predict)
print(f"Testing Error: {testing_error} (RMS: {testing_error**0.5})")

Training Error: 8.396083444368179 (RMS: 2.897599600422422)
Testing Error: 8.839515947385067 (RMS: 2.973132346093101)


In [51]:
%%timeit
OurLinearRegression().fit(X_train, y_train)
# Now, you can compare the time of our implementation with Scikit-Learn's. What is your conclusion?

14.3 µs ± 160 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [52]:
#TODO: Write your conclusion about your implementation's performance and training time

In [53]:
''' 
Training Time
Our implementation: 14.3 µs ± 160 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
scikit-learn: 310 µs ± 6.19 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)

Performance in terms of training error and testing error

Our implementation:
    Training Error: 8.396083444368179 (RMS: 2.897599600422422)
    Testing Error: 8.839515947385067 (RMS: 2.973132346093101)

scikit-learn:
    Training Error: 8.368261516136105 (RMS: 2.89279475873006)
    Testing Error: 8.78499323427013 (RMS: 2.9639489257188845)

Conclusion:
Our implementation is faster than scikit-learn's implementation. However, the performance of both implementations is almost the same.
'''

" \nTraining Time\nOur implementation: 14.4 µs ± 82.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)\nscikit-learn: 301 µs ± 3.14 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)\n\nPerformance in terms of training error and testing error\n\nOur implementation:\n    Training Error: 8.396083444368179 (RMS: 2.897599600422422)\n    Testing Error: 8.839515947385067 (RMS: 2.973132346093101)\n\nscikit-learn:\n    Training Error: 8.368261516136105 (RMS: 2.89279475873006)\n    Testing Error: 8.78499323427013 (RMS: 2.9639489257188845)\n\nConclusion:\nOur implementation is faster than scikit-learn's implementation. However, the performance of both implementations is almost the same.\n"